<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [2]:
import numpy as np
import pandas as pd
#import nltk
#from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.linear_model import LogisticRegression

from catboost import CatBoostClassifier
from catboost.text_processing import Tokenizer

from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score,confusion_matrix

## Подготовка

In [4]:
df = pd.read_csv('/datasets/toxic_comments.csv',index_col=0)
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [8]:
%%time

tr,te = train_test_split(df, test_size = .25, shuffle = True)
X = [ 'text' ] 
y = 'toxic'


CPU times: user 48.3 ms, sys: 6.62 ms, total: 54.9 ms
Wall time: 52.1 ms


## Обучение

In [4]:
tok = Tokenizer(
    lowercasing=True,
    separator_type='ByDelimiter',
    token_types=['Word']
)


In [29]:
%%time
params_grid ={
    'learning_rate':[0.3,.1,.03,.01],
    'auto_class_weights':['SqrtBalanced']
}
cbc = GridSearchCV( 
    CatBoostClassifier( eval_metric='F1',text_features = X,tokenizers=verbose= 100  ),
    params_grid,
    scoring='f1_weighted',
    cv= 4)  
cbc.fit(tr[X],tr[y])

0:	learn: 0.7384624	total: 198ms	remaining: 3m 17s
100:	learn: 0.8103162	total: 16.6s	remaining: 2m 27s
200:	learn: 0.8319522	total: 34.7s	remaining: 2m 17s
300:	learn: 0.8491065	total: 54.8s	remaining: 2m 7s
400:	learn: 0.8641802	total: 1m 12s	remaining: 1m 48s
500:	learn: 0.8795232	total: 1m 34s	remaining: 1m 33s
600:	learn: 0.8912642	total: 1m 55s	remaining: 1m 16s
700:	learn: 0.9044433	total: 2m 13s	remaining: 57s
800:	learn: 0.9147381	total: 2m 30s	remaining: 37.4s
900:	learn: 0.9239395	total: 2m 49s	remaining: 18.6s
999:	learn: 0.9310430	total: 3m 6s	remaining: 0us
0:	learn: 0.7455769	total: 179ms	remaining: 2m 58s
100:	learn: 0.8062791	total: 17.5s	remaining: 2m 35s
200:	learn: 0.8287581	total: 33.7s	remaining: 2m 13s
300:	learn: 0.8473597	total: 49.9s	remaining: 1m 55s
400:	learn: 0.8646014	total: 1m 6s	remaining: 1m 39s
500:	learn: 0.8802370	total: 1m 24s	remaining: 1m 24s
600:	learn: 0.8935904	total: 1m 42s	remaining: 1m 8s
700:	learn: 0.9065037	total: 2m	remaining: 51.6s
800

200:	learn: 0.7561296	total: 34.2s	remaining: 2m 16s
300:	learn: 0.7609823	total: 51.6s	remaining: 1m 59s
400:	learn: 0.7645473	total: 1m 8s	remaining: 1m 42s
500:	learn: 0.7685000	total: 1m 25s	remaining: 1m 25s
600:	learn: 0.7711226	total: 1m 41s	remaining: 1m 7s
700:	learn: 0.7725662	total: 1m 58s	remaining: 50.5s
800:	learn: 0.7743196	total: 2m 15s	remaining: 33.6s
900:	learn: 0.7764143	total: 2m 31s	remaining: 16.7s
999:	learn: 0.7781092	total: 2m 48s	remaining: 0us
0:	learn: 0.7400161	total: 164ms	remaining: 2m 43s
100:	learn: 0.7454060	total: 17.1s	remaining: 2m 31s
200:	learn: 0.7566346	total: 34.2s	remaining: 2m 15s
300:	learn: 0.7607385	total: 51.3s	remaining: 1m 59s
400:	learn: 0.7642849	total: 1m 8s	remaining: 1m 41s
500:	learn: 0.7678471	total: 1m 25s	remaining: 1m 24s
600:	learn: 0.7698833	total: 1m 41s	remaining: 1m 7s
700:	learn: 0.7719081	total: 1m 58s	remaining: 50.5s
800:	learn: 0.7741933	total: 2m 15s	remaining: 33.6s
900:	learn: 0.7759129	total: 2m 31s	remaining: 1

400:	learn: 0.8432555	total: 1m 8s	remaining: 1m 42s
500:	learn: 0.8469209	total: 1m 25s	remaining: 1m 25s
600:	learn: 0.8490692	total: 1m 42s	remaining: 1m 7s
700:	learn: 0.8515380	total: 1m 58s	remaining: 50.7s
800:	learn: 0.8528449	total: 2m 15s	remaining: 33.7s
900:	learn: 0.8540197	total: 2m 32s	remaining: 16.7s
999:	learn: 0.8552274	total: 2m 48s	remaining: 0us
0:	learn: 0.7954011	total: 153ms	remaining: 2m 33s
100:	learn: 0.8219296	total: 17.2s	remaining: 2m 32s
200:	learn: 0.8355294	total: 34.6s	remaining: 2m 17s
300:	learn: 0.8407723	total: 51.6s	remaining: 1m 59s
400:	learn: 0.8438556	total: 1m 8s	remaining: 1m 42s
500:	learn: 0.8473075	total: 1m 25s	remaining: 1m 25s
600:	learn: 0.8493549	total: 1m 41s	remaining: 1m 7s
700:	learn: 0.8509264	total: 1m 58s	remaining: 50.6s
800:	learn: 0.8527380	total: 2m 15s	remaining: 33.6s
900:	learn: 0.8546974	total: 2m 32s	remaining: 16.7s
999:	learn: 0.8561092	total: 2m 48s	remaining: 0us
0:	learn: 0.8021207	total: 175ms	remaining: 2m 54s

/home/ltz/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
16 fits failed out of a total of 48.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
16 fits failed with the following error:
Traceback (most recent call last):
  File "/home/ltz/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/ltz/anaconda3/lib/python3.9/site-packages/catboost/core.py", line 5131, in fit
    self._fit(X, y, cat_features, text_features, embedding_features, None, sample_weight, None, None, None, None, baseline, use_best_model,
  File "/home/ltz/anaconda3/lib/python3.9/site-packages/catboost/co

0:	learn: 0.7403711	total: 165ms	remaining: 2m 44s
100:	learn: 0.8078041	total: 17.9s	remaining: 2m 39s
200:	learn: 0.8252506	total: 35.2s	remaining: 2m 20s
300:	learn: 0.8402978	total: 52.7s	remaining: 2m 2s
400:	learn: 0.8530671	total: 1m 10s	remaining: 1m 44s
500:	learn: 0.8655247	total: 1m 27s	remaining: 1m 27s
600:	learn: 0.8763538	total: 1m 45s	remaining: 1m 10s
700:	learn: 0.8881198	total: 2m 3s	remaining: 52.5s
800:	learn: 0.8990210	total: 2m 21s	remaining: 35s
900:	learn: 0.9082995	total: 2m 38s	remaining: 17.4s
999:	learn: 0.9161650	total: 2m 56s	remaining: 0us
CPU times: user 7h 4min 25s, sys: 4h 40min 55s, total: 11h 45min 20s
Wall time: 1h 36min 35s


GridSearchCV(cv=4,
             estimator=<catboost.core.CatBoostClassifier object at 0x7f9c9a79c9d0>,
             param_grid={'auto_class_weights': ['SqrtBalanced', 'Balanced',
                                                None],
                         'learning_rate': [0.3, 0.1, 0.03, 0.01]},
             scoring='f1_weighted')

In [30]:
cbc.cv_results_

{'mean_fit_time': array([1.80591636e+02, 1.72052562e+02, 1.72840027e+02, 1.74259291e+02,
        1.76323581e+02, 1.72846996e+02, 1.74331292e+02, 1.75073971e+02,
        4.65340614e-02, 3.16201448e-02, 3.48017812e-02, 3.73852253e-02]),
 'std_fit_time': array([8.36674347e+00, 4.22584988e-01, 3.47478723e-01, 6.66180839e-01,
        2.09515558e+00, 2.46811626e-01, 3.44319653e-01, 3.87839272e-01,
        2.36616661e-02, 2.61194300e-04, 5.14900582e-03, 5.67147569e-03]),
 'mean_score_time': array([0.68075418, 0.57535881, 0.4171837 , 0.40378666, 0.61664939,
        0.54189628, 0.46334076, 0.49872804, 0.        , 0.        ,
        0.        , 0.        ]),
 'std_score_time': array([0.02235291, 0.03585024, 0.02386976, 0.01869129, 0.01651636,
        0.01609823, 0.01094381, 0.02445594, 0.        , 0.        ,
        0.        , 0.        ]),
 'param_auto_class_weights': masked_array(data=['SqrtBalanced', 'SqrtBalanced', 'SqrtBalanced',
                    'SqrtBalanced', 'Balanced', 'Balanced'

In [ ]:
pr = cbc.predict(te[X])

In [50]:
for metric in [f1_score,accuracy_score,precision_score,recall_score]:
    print(f"{metric.__name__}:{metric(te[y],pr)}\t")

f1_score:0.7194634335096092	
accuracy_score:0.9453833211962936	
precision_score:0.752563410685375	
recall_score:0.689152458611317	


In [75]:
model =  CatBoostClassifier( eval_metric='F1',eval_period=50,text_features = X,verbose= 100,n_estimators=500 , learning_rate=.03 )

TypeError: __init__() got an unexpected keyword argument 'eval_period'

In [48]:
model.fit(tr[X],tr[y])
pr = model.predict(te[X])

0:	learn: 0.7403711	total: 175ms	remaining: 14m 35s
100:	learn: 0.8078041	total: 17.8s	remaining: 14m 25s
200:	learn: 0.8252506	total: 35.4s	remaining: 14m 5s
300:	learn: 0.8402978	total: 53.2s	remaining: 13m 50s
400:	learn: 0.8530671	total: 1m 13s	remaining: 13m 58s
500:	learn: 0.8655247	total: 1m 32s	remaining: 13m 46s
600:	learn: 0.8763538	total: 1m 51s	remaining: 13m 37s
700:	learn: 0.8881198	total: 2m 11s	remaining: 13m 25s
800:	learn: 0.8990210	total: 2m 29s	remaining: 13m 2s
900:	learn: 0.9082995	total: 2m 47s	remaining: 12m 43s
1000:	learn: 0.9163515	total: 3m 7s	remaining: 12m 28s
1100:	learn: 0.9234957	total: 3m 25s	remaining: 12m 7s
1200:	learn: 0.9300662	total: 3m 43s	remaining: 11m 47s
1300:	learn: 0.9353959	total: 4m 2s	remaining: 11m 28s
1400:	learn: 0.9398499	total: 4m 19s	remaining: 11m 7s
1500:	learn: 0.9444344	total: 4m 39s	remaining: 10m 50s
1600:	learn: 0.9483924	total: 4m 57s	remaining: 10m 30s
1700:	learn: 0.9507273	total: 5m 15s	remaining: 10m 12s
1800:	learn: 0

In [49]:
confusion_matrix(te[y],pr)

array([[34859,   917],
       [ 1258,  2789]])

In [59]:
pb = model.predict_proba(te[X])

In [87]:
model.get_evals_result().keys()

dict_keys(['learn'])

In [73]:
for i in range(30,70,2):
    print(i, round( f1_score( te[y], (pb>(i/100))[:,1].astype(int)) ,4 ) ) 

30 0.7078
32 0.7098
34 0.7108
36 0.7114
38 0.7135
40 0.7148
42 0.7153
44 0.7159
46 0.7161
48 0.7185
50 0.7195
52 0.7175
54 0.7166
56 0.717
58 0.7177
60 0.7188
62 0.7181
64 0.7154
66 0.7137
68 0.7121


## Выводы

## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Данные загружены и подготовлены
- [ ]  Модели обучены
- [ ]  Значение метрики *F1* не меньше 0.75
- [ ]  Выводы написаны